# PostreSQL

## Install PostgreSQL

In [1]:
!apt install postgresql postgresql-contrib &>log

In [2]:
!service postgresql start

 * Starting PostgreSQL 14 database server
   ...done.


In [3]:
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"

CREATE ROLE


## Set PotgreSQL connection

In [4]:
%load_ext sql

In [5]:
%config SqlMagic.feedback=False

In [6]:
%config SqlMagic.autopandas=True

In [7]:
%sql postgresql+psycopg2://@/postgres

# XLA

In [9]:
!pip install torch~=2.0.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.9/162.9 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 5.3 MB/s eta 0:00:00
  Attempting uninstall: uritemplate
    Found existing installation: uritemplate 4.1.1
    Uninstalling uritemplate-4.1.1:
      Successfully uninstalled uritemplate-4.1.1
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 2.11.1
    Uninstalling google-api-core-2.11.1:
      Successfully uninstalled google-api-core-2.11.1
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conf

In [11]:
import torch_xla
import torch_xla.core.xla_model as xm

# Optuna simple distributed

## Install Optuna

In [ ]:
%pip install optuna

## Import Optuna

In [13]:
import optuna

## Define objective function

In [14]:
def objective(trial):
   x = trial.suggest_uniform('x', -10, 10)
   return (x - 2) ** 2

## Create a study

In [15]:
study = optuna.create_study(study_name="distributed-example", storage="postgresql+psycopg2://@/postgres")

[I 2023-09-01 17:30:14,327] A new study created in RDB with name: distributed-example


## Define a parallel distribution function and load created study to every XLA device

In [16]:
def para_fn(device):
  loaded_study = optuna.load_study(study_name="distributed-example", storage="postgresql+psycopg2://@/postgres")
  loaded_study.optimize(objective, n_trials=2)
  params = loaded_study.best_params
  print('\rDevice {}\t Best value is: {}'.format(device, params))

## Wrap a parallel distribution function into a run fuction and launch it on XLA cores

In [17]:
from joblib import Parallel, delayed

def run(fold):
    device = xm.xla_device(fold + 1)
    para_fn(device)
Parallel(n_jobs=8, backend="threading")(delayed(run)(i) for i in range(8))

<ipython-input-14-8163ab530af6>:2: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  x = trial.suggest_uniform('x', -10, 10)
[I 2023-09-01 17:30:38,952] Trial 0 finished with value: 0.7687857186901794 and parameters: {'x': 2.8768042647536447}. Best is trial 2 with value: 0.1466573575614657.
[I 2023-09-01 17:30:38,964] Trial 2 finished with value: 0.1466573575614657 and parameters: {'x': 1.6170413108944182}. Best is trial 2 with value: 0.1466573575614657.
[I 2023-09-01 17:30:38,984] Trial 4 finished with value: 0.20183881431615192 and parameters: {'x': 1.550735251420555}. Best is trial 2 with value: 0.1466573575614657.
[I 2023-09-01 17:30:38,989] Trial 5 finished with value: 7.04566794369932 and parameters: {'x': 4.654367710717436}. Best is trial 2 with value: 0.1466573575614657.
[I 2023-09-01 17:30:38,996] Trial 1 finished with value: 59.40743369141

Device xla:3	 Best value is: {'x': 1.6170413108944182}
Device xla:8	 Best value is: {'x': 1.6170413108944182}
Device xla:2	 Best value is: {'x': 1.6170413108944182}
Device xla:6	 Best value is: {'x': 1.6170413108944182}
Device xla:4	 Best value is: {'x': 1.6170413108944182}
Device xla:7	 Best value is: {'x': 1.6170413108944182}
Device xla:1	 Best value is: {'x': 1.6170413108944182}
Device xla:5	 Best value is: {'x': 1.6170413108944182}


[None, None, None, None, None, None, None, None]

In [18]:
df = %sql SELECT * FROM pg_catalog.pg_tables
df

 * postgresql+psycopg2://@/postgres


,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers,rowsecurity
0,public,version_info,root,None,True,False,False,False
1,public,studies,root,None,True,False,True,False
2,public,study_directions,root,None,True,False,True,False
3,public,study_user_attributes,root,None,True,False,True,False
4,public,study_system_attributes,root,None,True,False,True,False
...,...,...,...,...,...,...,...,...
74,pg_catalog,pg_largeobject,postgres,None,True,False,False,False
75,information_schema,sql_parts,postgres,None,False,False,False,False
76,information_schema,sql_implementation_info,postgres,None,False,False,False,False
77,information_schema,sql_features,postgres,None,False,False,False,False


In [20]:
loaded_study = optuna.load_study(study_name="distributed-example", storage="postgresql+psycopg2://@/postgres")

loaded_study.optimize(objective, n_trials=5)
loaded_study.best_params

<ipython-input-14-8163ab530af6>:2: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  x = trial.suggest_uniform('x', -10, 10)
[I 2023-09-01 17:31:04,752] Trial 16 finished with value: 51.9160097986074 and parameters: {'x': 9.205276524784278}. Best is trial 2 with value: 0.1466573575614657.
[I 2023-09-01 17:31:04,966] Trial 17 finished with value: 3.6128943971861562 and parameters: {'x': 0.09923846914291845}. Best is trial 2 with value: 0.1466573575614657.
[I 2023-09-01 17:31:05,045] Trial 18 finished with value: 5.206980420229954 and parameters: {'x': -0.28188089527695404}. Best is trial 2 with value: 0.1466573575614657.
[I 2023-09-01 17:31:05,125] Trial 19 finished with value: 5.121540722752247 and parameters: {'x': -0.2630821290338199}. Best is trial 2 with value: 0.1466573575614657.
[I 2023-09-01 17:31:05,201] Trial 20 finished with value: 1.73982

{'x': 1.6170413108944182}

In [21]:
study.optimize(objective, n_trials=5)
study.best_params

<ipython-input-14-8163ab530af6>:2: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  x = trial.suggest_uniform('x', -10, 10)
[I 2023-09-01 17:31:08,890] Trial 21 finished with value: 0.4358065214367236 and parameters: {'x': 1.33984356290594}. Best is trial 2 with value: 0.1466573575614657.
[I 2023-09-01 17:31:08,965] Trial 22 finished with value: 17.919815888223717 and parameters: {'x': -2.2331803514879587}. Best is trial 2 with value: 0.1466573575614657.
[I 2023-09-01 17:31:09,041] Trial 23 finished with value: 0.47173644489352967 and parameters: {'x': 1.3131692749348427}. Best is trial 2 with value: 0.1466573575614657.
[I 2023-09-01 17:31:09,117] Trial 24 finished with value: 18.331893517637816 and parameters: {'x': -2.281576055337312}. Best is trial 2 with value: 0.1466573575614657.
[I 2023-09-01 17:31:09,192] Trial 25 finished with value: 0.0751

{'x': 1.7259506014341848}

## Optuna with dataset in a parameter

In [22]:
import os

import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

DEVICE = torch.device("cpu")
BATCHSIZE = 128
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
LOG_INTERVAL = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10

train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
valid_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

100%|██████████| 26421880/26421880 [00:01<00:00, 15018323.69it/s]


Extracting /content/FashionMNIST/raw/train-images-idx3-ubyte.gz to /content/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 265988.17it/s]


Extracting /content/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /content/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5073205.72it/s]


Extracting /content/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /content/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 10954985.79it/s]

Extracting /content/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /content/FashionMNIST/raw



In [23]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

In [24]:
class ObjectivePytorch(object):
    def __init__(self, train_loader, valid_loader):
        self.train_loader = train_loader
        self.valid_loader = valid_loader

    def __call__(self, trial):
        # Generate the model.
        model = define_model(trial).to(DEVICE)

        # Generate the optimizers.
        optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
        lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
        optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

        # Training of the model.
        for epoch in range(EPOCHS):
            model.train()
            for batch_idx, (data, target) in enumerate(train_loader):
                # Limiting training data for faster epochs.
                if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                    break

                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

                optimizer.zero_grad()
                output = model(data)
                loss = F.nll_loss(output, target)
                loss.backward()
                optimizer.step()
            # Validation of the model.
            model.eval()
            correct = 0
            with torch.no_grad():
                for batch_idx, (data, target) in enumerate(valid_loader):
                    # Limiting validation data.
                    if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                        break
                    data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                    output = model(data)
                    # Get the index of the max log-probability.
                    pred = output.argmax(dim=1, keepdim=True)
                    correct += pred.eq(target.view_as(pred)).sum().item()

            accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

            trial.report(accuracy, epoch)

            # Handle pruning based on the intermediate value.
            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        return accuracy

In [25]:
objective_pytorch = ObjectivePytorch(train_loader, valid_loader)
study = optuna.create_study(direction="maximize", study_name="distributed-example-advanced-in-memory-dataset", storage="postgresql+psycopg2://@/postgres", load_if_exists=True)

[I 2023-09-01 17:31:31,341] A new study created in RDB with name: distributed-example-advanced-in-memory-dataset


In [26]:
def para_fn(device):
  loaded_study = optuna.load_study(study_name="distributed-example-advanced-in-memory-dataset", storage="postgresql+psycopg2://@/postgres")

  loaded_study.optimize(objective_pytorch, n_trials=2)

  trial = loaded_study.best_trial
  params = loaded_study.best_params
  pruned_trials = loaded_study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
  complete_trials = loaded_study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

  print('\rDevice {}\t Best trial is: {}\t Best params are: {}\t Finished trials: {}\t Pruned trials: {}\t Complete trials: {}'
  .format(device, trial.values[0], params, len(loaded_study.trials), pruned_trials, complete_trials))

In [27]:
from joblib import Parallel, delayed

def run(fold):
    device = xm.xla_device(fold + 1)
    DEVICE = device
    para_fn(device)

Parallel(n_jobs=8, backend="threading")(delayed(run)(i) for i in range(8))

[I 2023-09-01 17:33:09,623] Trial 7 finished with value: 0.71015625 and parameters: {'n_layers': 1, 'n_units_l0': 19, 'dropout_l0': 0.23718902248648108, 'optimizer': 'SGD', 'lr': 0.028647724981487942}. Best is trial 7 with value: 0.71015625.
[I 2023-09-01 17:33:10,517] Trial 3 finished with value: 0.6328125 and parameters: {'n_layers': 1, 'n_units_l0': 8, 'dropout_l0': 0.4915431091499663, 'optimizer': 'SGD', 'lr': 0.014897722552336286}. Best is trial 7 with value: 0.71015625.
[I 2023-09-01 17:33:12,029] Trial 4 finished with value: 0.31875 and parameters: {'n_layers': 1, 'n_units_l0': 10, 'dropout_l0': 0.4809329639591433, 'optimizer': 'Adam', 'lr': 9.78315782444798e-05}. Best is trial 7 with value: 0.71015625.
[I 2023-09-01 17:33:13,381] Trial 6 finished with value: 0.2859375 and parameters: {'n_layers': 3, 'n_units_l0': 5, 'dropout_l0': 0.227301731033748, 'n_units_l1': 14, 'dropout_l1': 0.2628317847996494, 'n_units_l2': 110, 'dropout_l2': 0.40007786012259283, 'optimizer': 'RMSprop', '

Device xla:8	 Best trial is: 0.7921875	 Best params are: {'n_layers': 1, 'n_units_l0': 122, 'dropout_l0': 0.22603580095074804, 'optimizer': 'RMSprop', 'lr': 0.010101852213778628}	 Finished trials: 16	 Pruned trials: [FrozenTrial(number=0, state=TrialState.PRUNED, values=[0.47109375], datetime_start=datetime.datetime(2023, 9, 1, 17, 31, 38, 380959), datetime_complete=datetime.datetime(2023, 9, 1, 17, 33, 22, 100050), params={'n_layers': 3, 'n_units_l0': 128, 'dropout_l0': 0.38075337002285287, 'n_units_l1': 27, 'dropout_l1': 0.41424000584468035, 'n_units_l2': 118, 'dropout_l2': 0.4670267529469203, 'optimizer': 'Adam', 'lr': 2.3124826261365857e-05}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.1734375, 1: 0.22265625, 2: 0.2921875, 3: 0.34765625, 4: 0.4390625, 5: 0.42890625, 6: 0.4421875, 7: 0.4609375, 8: 0.50234375, 9: 0.47109375}, distributions={'n_layers': IntDistribution(high=3, log=False, low=1, step=1), 'n_units_l0': IntDistribution(high=128, log=False, low=4, step=1), '

[I 2023-09-01 17:33:26,334] Trial 14 pruned. 


Device xla:3	 Best trial is: 0.7921875	 Best params are: {'n_layers': 1, 'n_units_l0': 122, 'dropout_l0': 0.22603580095074804, 'optimizer': 'RMSprop', 'lr': 0.010101852213778628}	 Finished trials: 16	 Pruned trials: [FrozenTrial(number=0, state=TrialState.PRUNED, values=[0.47109375], datetime_start=datetime.datetime(2023, 9, 1, 17, 31, 38, 380959), datetime_complete=datetime.datetime(2023, 9, 1, 17, 33, 22, 100050), params={'n_layers': 3, 'n_units_l0': 128, 'dropout_l0': 0.38075337002285287, 'n_units_l1': 27, 'dropout_l1': 0.41424000584468035, 'n_units_l2': 118, 'dropout_l2': 0.4670267529469203, 'optimizer': 'Adam', 'lr': 2.3124826261365857e-05}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.1734375, 1: 0.22265625, 2: 0.2921875, 3: 0.34765625, 4: 0.4390625, 5: 0.42890625, 6: 0.4421875, 7: 0.4609375, 8: 0.50234375, 9: 0.47109375}, distributions={'n_layers': IntDistribution(high=3, log=False, low=1, step=1), 'n_units_l0': IntDistribution(high=128, log=False, low=4, step=1), '

[I 2023-09-01 17:33:30,342] Trial 15 pruned. 
[I 2023-09-01 17:33:30,376] Trial 10 pruned. 


...

[I 2023-09-01 17:34:08,842] Trial 12 finished with value: 0.6875 and parameters: {'n_layers': 1, 'n_units_l0': 56, 'dropout_l0': 0.39023607933722204, 'optimizer': 'RMSprop', 'lr': 0.00015529071137695516}. Best is trial 2 with value: 0.7921875.


Device xla:6	 Best trial is: 0.7921875	 Best params are: {'n_layers': 1, 'n_units_l0': 122, 'dropout_l0': 0.22603580095074804, 'optimizer': 'RMSprop', 'lr': 0.010101852213778628}	 Finished trials: 16	 Pruned trials: [FrozenTrial(number=0, state=TrialState.PRUNED, values=[0.47109375], datetime_start=datetime.datetime(2023, 9, 1, 17, 31, 38, 380959), datetime_complete=datetime.datetime(2023, 9, 1, 17, 33, 22, 100050), params={'n_layers': 3, 'n_units_l0': 128, 'dropout_l0': 0.38075337002285287, 'n_units_l1': 27, 'dropout_l1': 0.41424000584468035, 'n_units_l2': 118, 'dropout_l2': 0.4670267529469203, 'optimizer': 'Adam', 'lr': 2.3124826261365857e-05}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.1734375, 1: 0.22265625, 2: 0.2921875, 3: 0.34765625, 4: 0.4390625, 5: 0.42890625, 6: 0.4421875, 7: 0.4609375, 8: 0.50234375, 9: 0.47109375}, distributions={'n_layers': IntDistribution(high=3, log=False, low=1, step=1), 'n_units_l0': IntDistribution(high=128, log=False, low=4, step=1), '

[I 2023-09-01 17:34:09,119] Trial 13 finished with value: 0.74375 and parameters: {'n_layers': 1, 'n_units_l0': 103, 'dropout_l0': 0.4657500036263731, 'optimizer': 'SGD', 'lr': 0.03477831735802452}. Best is trial 2 with value: 0.7921875.


Device xla:1	 Best trial is: 0.7921875	 Best params are: {'n_layers': 1, 'n_units_l0': 122, 'dropout_l0': 0.22603580095074804, 'optimizer': 'RMSprop', 'lr': 0.010101852213778628}	 Finished trials: 16	 Pruned trials: [FrozenTrial(number=0, state=TrialState.PRUNED, values=[0.47109375], datetime_start=datetime.datetime(2023, 9, 1, 17, 31, 38, 380959), datetime_complete=datetime.datetime(2023, 9, 1, 17, 33, 22, 100050), params={'n_layers': 3, 'n_units_l0': 128, 'dropout_l0': 0.38075337002285287, 'n_units_l1': 27, 'dropout_l1': 0.41424000584468035, 'n_units_l2': 118, 'dropout_l2': 0.4670267529469203, 'optimizer': 'Adam', 'lr': 2.3124826261365857e-05}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.1734375, 1: 0.22265625, 2: 0.2921875, 3: 0.34765625, 4: 0.4390625, 5: 0.42890625, 6: 0.4421875, 7: 0.4609375, 8: 0.50234375, 9: 0.47109375}, distributions={'n_layers': IntDistribution(high=3, log=False, low=1, step=1), 'n_units_l0': IntDistribution(high=128, log=False, low=4, step=1), '

[None, None, None, None, None, None, None, None]

# Optuna advanced
### taken from https://github.com/optuna/optuna/blob/master/examples/pytorch/pytorch_simple.py

## Imports and params

In [28]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data
from torchvision import datasets
from torchvision import transforms

import optuna
from optuna.trial import TrialState

DEVICE = torch.device("xla")
BATCHSIZE = 64
CLASSES = 10
DIR = os.getcwd()
EPOCHS = 10
LOG_INTERVAL = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 20
N_VALID_EXAMPLES = BATCHSIZE * 5

## Layers & dropout optimization

In [29]:
def define_model(trial):
    # We optimize the number of layers, hidden units and dropout ratio in each layer.
    n_layers = trial.suggest_int("n_layers", 1, 3)
    layers = []

    in_features = 28 * 28
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, CLASSES))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

## Dataset

In [30]:
def get_mnist():
    # Load FashionMNIST dataset.
    train_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=True, download=True, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )
    valid_loader = torch.utils.data.DataLoader(
        datasets.FashionMNIST(DIR, train=False, transform=transforms.ToTensor()),
        batch_size=BATCHSIZE,
        shuffle=True,
    )

    return train_loader, valid_loader

## Objective

In [31]:
def objective(trial):

    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the FashionMNIST dataset.
    train_loader, valid_loader = get_mnist()

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break
            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()
        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)
        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

## Study

In [32]:
study = optuna.create_study(direction="maximize", study_name="distributed-example-advanced", storage="postgresql+psycopg2://@/postgres", load_if_exists=True)

[I 2023-09-01 17:34:53,965] A new study created in RDB with name: distributed-example-advanced


## Parallel distribution

In [33]:
def para_fn(device):
  loaded_study = optuna.load_study(study_name="distributed-example-advanced", storage="postgresql+psycopg2://@/postgres")

  loaded_study.optimize(objective, n_trials=2)

  trial = loaded_study.best_trial
  params = loaded_study.best_params
  pruned_trials = loaded_study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
  complete_trials = loaded_study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

  print('\rDevice {}\t Best trial is: {}\t Best params are: {}\t Finished trials: {}\t Pruned trials: {}\t Complete trials: {}'
  .format(device, trial.values[0], params, len(loaded_study.trials), pruned_trials, complete_trials))

In [34]:
from joblib import Parallel, delayed

def run(fold):
    device = xm.xla_device(fold + 1)
    DEVICE = device
    para_fn(device)

Parallel(n_jobs=8, backend="threading")(delayed(run)(i) for i in range(8))

[I 2023-09-01 17:58:12,702] Trial 0 finished with value: 0.13125 and parameters: {'n_layers': 2, 'n_units_l0': 88, 'dropout_l0': 0.38544392176223574, 'n_units_l1': 53, 'dropout_l1': 0.3674667972416941, 'optimizer': 'SGD', 'lr': 0.0008110501555179943}. Best is trial 0 with value: 0.13125.
[I 2023-09-01 17:59:33,419] Trial 4 finished with value: 0.6875 and parameters: {'n_layers': 2, 'n_units_l0': 70, 'dropout_l0': 0.21762821136488056, 'n_units_l1': 114, 'dropout_l1': 0.216719572488067, 'optimizer': 'SGD', 'lr': 0.06751739082213563}. Best is trial 4 with value: 0.6875.
[I 2023-09-01 18:00:28,451] Trial 1 finished with value: 0.678125 and parameters: {'n_layers': 1, 'n_units_l0': 108, 'dropout_l0': 0.22655547770644138, 'optimizer': 'RMSprop', 'lr': 5.01903112447575e-05}. Best is trial 4 with value: 0.6875.
[I 2023-09-01 18:01:05,995] Trial 5 finished with value: 0.675 and parameters: {'n_layers': 1, 'n_units_l0': 70, 'dropout_l0': 0.2125313712791924, 'optimizer': 'RMSprop', 'lr': 0.035055

Device xla:6	 Best trial is: 0.75625	 Best params are: {'n_layers': 2, 'n_units_l0': 106, 'dropout_l0': 0.4131347793374167, 'n_units_l1': 55, 'dropout_l1': 0.212257947140673, 'optimizer': 'RMSprop', 'lr': 0.005008655700954662}	 Finished trials: 15	 Pruned trials: [FrozenTrial(number=6, state=TrialState.PRUNED, values=[0.259375], datetime_start=datetime.datetime(2023, 9, 1, 17, 34, 58, 205690), datetime_complete=datetime.datetime(2023, 9, 1, 18, 8, 38, 573680), params={'n_layers': 3, 'n_units_l0': 57, 'dropout_l0': 0.46140059157792346, 'n_units_l1': 62, 'dropout_l1': 0.4880138355177629, 'n_units_l2': 64, 'dropout_l2': 0.37749645863265213, 'optimizer': 'RMSprop', 'lr': 6.566113322809912e-05}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.2, 1: 0.20625, 2: 0.278125, 3: 0.228125, 4: 0.315625, 5: 0.35625, 6: 0.321875, 7: 0.40625, 8: 0.259375}, distributions={'n_layers': IntDistribution(high=3, log=False, low=1, step=1), 'n_units_l0': IntDistribution(high=128, log=False, low=4, s

[I 2023-09-01 18:11:01,352] Trial 13 pruned. 


Device xla:1	 Best trial is: 0.75625	 Best params are: {'n_layers': 2, 'n_units_l0': 106, 'dropout_l0': 0.4131347793374167, 'n_units_l1': 55, 'dropout_l1': 0.212257947140673, 'optimizer': 'RMSprop', 'lr': 0.005008655700954662}	 Finished trials: 16	 Pruned trials: [FrozenTrial(number=6, state=TrialState.PRUNED, values=[0.259375], datetime_start=datetime.datetime(2023, 9, 1, 17, 34, 58, 205690), datetime_complete=datetime.datetime(2023, 9, 1, 18, 8, 38, 573680), params={'n_layers': 3, 'n_units_l0': 57, 'dropout_l0': 0.46140059157792346, 'n_units_l1': 62, 'dropout_l1': 0.4880138355177629, 'n_units_l2': 64, 'dropout_l2': 0.37749645863265213, 'optimizer': 'RMSprop', 'lr': 6.566113322809912e-05}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.2, 1: 0.20625, 2: 0.278125, 3: 0.228125, 4: 0.315625, 5: 0.35625, 6: 0.321875, 7: 0.40625, 8: 0.259375}, distributions={'n_layers': IntDistribution(high=3, log=False, low=1, step=1), 'n_units_l0': IntDistribution(high=128, log=False, low=4, s

[I 2023-09-01 18:11:15,996] Trial 11 pruned. 


...

[I 2023-09-01 18:31:39,122] Trial 8 finished with value: 0.6875 and parameters: {'n_layers': 3, 'n_units_l0': 49, 'dropout_l0': 0.36236404666585853, 'n_units_l1': 125, 'dropout_l1': 0.4208604251617582, 'n_units_l2': 41, 'dropout_l2': 0.490725851886296, 'optimizer': 'RMSprop', 'lr': 0.00189073705176855}. Best is trial 9 with value: 0.803125.


Device xla:2	 Best trial is: 0.803125	 Best params are: {'n_layers': 1, 'n_units_l0': 121, 'dropout_l0': 0.4390613254642145, 'optimizer': 'RMSprop', 'lr': 0.0004654633567334438}	 Finished trials: 16	 Pruned trials: [FrozenTrial(number=6, state=TrialState.PRUNED, values=[0.259375], datetime_start=datetime.datetime(2023, 9, 1, 17, 34, 58, 205690), datetime_complete=datetime.datetime(2023, 9, 1, 18, 8, 38, 573680), params={'n_layers': 3, 'n_units_l0': 57, 'dropout_l0': 0.46140059157792346, 'n_units_l1': 62, 'dropout_l1': 0.4880138355177629, 'n_units_l2': 64, 'dropout_l2': 0.37749645863265213, 'optimizer': 'RMSprop', 'lr': 6.566113322809912e-05}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.2, 1: 0.20625, 2: 0.278125, 3: 0.228125, 4: 0.315625, 5: 0.35625, 6: 0.321875, 7: 0.40625, 8: 0.259375}, distributions={'n_layers': IntDistribution(high=3, log=False, low=1, step=1), 'n_units_l0': IntDistribution(high=128, log=False, low=4, step=1), 'dropout_l0': FloatDistribution(high=0.5,

[I 2023-09-01 18:32:49,129] Trial 14 finished with value: 0.75 and parameters: {'n_layers': 2, 'n_units_l0': 59, 'dropout_l0': 0.2929078580049342, 'n_units_l1': 74, 'dropout_l1': 0.2469122440347837, 'optimizer': 'RMSprop', 'lr': 0.000754065315933356}. Best is trial 9 with value: 0.803125.


Device xla:8	 Best trial is: 0.803125	 Best params are: {'n_layers': 1, 'n_units_l0': 121, 'dropout_l0': 0.4390613254642145, 'optimizer': 'RMSprop', 'lr': 0.0004654633567334438}	 Finished trials: 16	 Pruned trials: [FrozenTrial(number=6, state=TrialState.PRUNED, values=[0.259375], datetime_start=datetime.datetime(2023, 9, 1, 17, 34, 58, 205690), datetime_complete=datetime.datetime(2023, 9, 1, 18, 8, 38, 573680), params={'n_layers': 3, 'n_units_l0': 57, 'dropout_l0': 0.46140059157792346, 'n_units_l1': 62, 'dropout_l1': 0.4880138355177629, 'n_units_l2': 64, 'dropout_l2': 0.37749645863265213, 'optimizer': 'RMSprop', 'lr': 6.566113322809912e-05}, user_attrs={}, system_attrs={}, intermediate_values={0: 0.2, 1: 0.20625, 2: 0.278125, 3: 0.228125, 4: 0.315625, 5: 0.35625, 6: 0.321875, 7: 0.40625, 8: 0.259375}, distributions={'n_layers': IntDistribution(high=3, log=False, low=1, step=1), 'n_units_l0': IntDistribution(high=128, log=False, low=4, step=1), 'dropout_l0': FloatDistribution(high=0.5,

[None, None, None, None, None, None, None, None]

## Further finetuning

In [35]:
fix_optimizer = {}
fix_optimizer['optimizer'] = study.best_params['optimizer']
study.sampler = optuna.samplers.PartialFixedSampler(fix_optimizer, study.sampler)
study.optimize(objective, n_trials=5)

<ipython-input-35-458a4a9edb07>:3: ExperimentalWarning: PartialFixedSampler is experimental (supported from v2.4.0). The interface can change in the future.
  study.sampler = optuna.samplers.PartialFixedSampler(fix_optimizer, study.sampler)
[I 2023-09-01 18:37:46,843] Trial 16 finished with value: 0.79375 and parameters: {'n_layers': 1, 'n_units_l0': 124, 'dropout_l0': 0.4910243515418116, 'optimizer': 'RMSprop', 'lr': 0.0003375550152258889}. Best is trial 9 with value: 0.803125.
[I 2023-09-01 18:43:31,488] Trial 17 finished with value: 0.7625 and parameters: {'n_layers': 1, 'n_units_l0': 122, 'dropout_l0': 0.48835733594152314, 'optimizer': 'RMSprop', 'lr': 0.0004000253932057074}. Best is trial 9 with value: 0.803125.
[I 2023-09-01 18:50:09,282] Trial 18 finished with value: 0.696875 and parameters: {'n_layers': 1, 'n_units_l0': 123, 'dropout_l0': 0.4932086029672041, 'optimizer': 'RMSprop', 'lr': 0.00024096186704898929}. Best is trial 9 with value: 0.803125.
[I 2023-09-01 18:51:06,936] 

In [36]:
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
  print("    {}: {}".format(key, value))

Best trial:
  Value:  0.803125
  Params: 
    n_layers: 1
    n_units_l0: 121
    dropout_l0: 0.4390613254642145
    optimizer: RMSprop
    lr: 0.0004654633567334438


In [37]:
study.best_trial.params.items()

dict_items([('n_layers', 1), ('n_units_l0', 121), ('dropout_l0', 0.4390613254642145), ('optimizer', 'RMSprop'), ('lr', 0.0004654633567334438)])

## Visualizations

In [38]:
optuna.visualization.plot_intermediate_values(study)

In [39]:
optuna.visualization.plot_param_importances(study)

In [40]:
optuna.visualization.plot_optimization_history(study)

In [41]:
optuna.visualization.plot_slice(study)

In [42]:
optuna.visualization.plot_slice(study, ['optimizer'])

In [43]:
optuna.visualization.plot_contour(study, ['lr', 'optimizer'])